In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

import re
import string
from collections import Counter
import nltk

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.cluster import KMeans, DBSCAN
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

import gensim

%matplotlib inline

In [2]:
df = pd.read_csv('df1csv.csv')
df.head()

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
0,OPERATIONAL,"{'lat': 20.6939009, 'lng': -105.2742384}",https://maps.gstatic.com/mapfiles/place_api/ic...,casa LAS PIÑAS,{'open_now': False},"[{'height': 640, 'html_attributions': ['<a hre...",ChIJIeVYvYNGIYQRh-9bPVK3fCU,"{'compound_code': 'MPVG+H8 Jarretaderas, Nayar...",4.6,ChIJIeVYvYNGIYQRh-9bPVK3fCU,...,casa LAS PIÑAS,4.6,"[{'author_name': 'Jovita Garcia Fernandez', 'a...",8.0,https://www.facebook.com/departamentos.laspinas/,"[5, 5, 5, 5, 5]","['An excellent place has all the comforts', 'I...",NaN,20.693901,-105.274238
1,OPERATIONAL,"{'lat': 20.6920731, 'lng': -105.2747303}",https://maps.gstatic.com/mapfiles/place_api/ic...,Villas Victoria Confort Jarretaderas,{'open_now': True},NaN,ChIJdSKPg3xHIYQRlZ0pmfgVPiQ,"{'compound_code': 'MPRG+R4 Jarretaderas, Nayar...",5.0,ChIJdSKPg3xHIYQRlZ0pmfgVPiQ,...,Villas Victoria Confort Jarretaderas,5.0,"[{'author_name': 'anahi chavez', 'author_url':...",1.0,https://www.villasvictoriacomfort.com/,[5],"['It was more than I expected! Cheap, spacious...",NaN,20.692073,-105.274730
2,OPERATIONAL,"{'lat': 20.6939068, 'lng': -105.2763239}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Posada Alejandra,NaN,"[{'height': 349, 'html_attributions': ['<a hre...",ChIJB2Yha4NGIYQRE-GTtF_VSAU,"{'compound_code': 'MPVF+HF Jarretaderas, Nayar...",4.0,ChIJB2Yha4NGIYQRE-GTtF_VSAU,...,Hotel Posada Alejandra,4.0,"[{'author_name': 'Juan Carlos Lopez', 'author_...",54.0,https://www.google.com/url?sa=t&source=web&rct...,"[3, 4, 3, 3, 5]","[""Just for one night it's alright"", 'Discreet'...",NaN,20.693907,-105.276324
3,OPERATIONAL,"{'lat': 20.6241471, 'lng': -105.2243218}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Azul Romero,{'open_now': True},"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJFW5JHF5FIYQRogsYTW-KvMI,"{'compound_code': 'JQFG+M7 Puerto Vallarta, Ja...",3.9,ChIJFW5JHF5FIYQRogsYTW-KvMI,...,Hotel Azul Romero,3.9,"[{'author_name': 'Al Reust', 'author_url': 'ht...",11.0,http://www.azulromero.com/,"[3, 4, 5, 4, 1]","['This is interesting, as it appears to be be ...",NaN,20.624147,-105.224322
4,OPERATIONAL,"{'lat': 20.6225929, 'lng': -105.2232518}",https://maps.gstatic.com/mapfiles/place_api/ic...,Vacational Apartment Apartamento Vacacional De...,NaN,"[{'height': 1052, 'html_attributions': ['<a hr...",ChIJ9321RV5FIYQRwnf0qJkeIqU,"{'compound_code': 'JQFG+2M Puerto Vallarta, Ja...",4.8,ChIJ9321RV5FIYQRwnf0qJkeIqU,...,Vacational Apartment Apartamento Vacacional De...,4.8,"[{'author_name': 'Daniel Chilldres', 'author_u...",11.0,https://www.airbnb.mx/rooms/23351610,"[5, 4, 5, 5, 5]","['I loved it here!', 'The place is clean, plea...",NaN,20.622593,-105.223252


In [3]:
df.iloc[6].all_reviews

"['Aight', 'Bad attitude of service offensive discrimination without giving reason or motive I definitely no longer recommend it', 'The net is very bad to rest because the man who takes care of the television at night makes the television very loud and does not let you sleep all morning.', 'An inexpensive place does not have the required amenities but is passable', 'Something good']"

In [4]:
remove_n = lambda x: re.sub('\\n',' ',x)
remove_singlen = lambda x: re.sub('\n',' ',x)
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

df['all_reviews'] = df.all_reviews.map(remove_n).map(punc_lower).map(alphanumeric)

In [5]:
df.iloc[3].all_reviews

'  this is interesting  as it appears to be be under new ownership  it has a new paint job on the front  i have not taken time to speak with the new managers  in the past few years i know of people who have did extended vacations as the prices were reasonable  access in the local community offers many things  i live across the street    there is a bus route to centro up the hill and return from centro down the hill  they have a few rooms for family or long term  i have seen buses unload and many people take over the hotel in the past  time will tell     comfortable and decent  it is   minutes from the beach nthe water heater does not work and neither does the internet  but good for spending the night    very good cows in guadalajara    very good    change of administration and they did not make the deposit valid according to why the past administration did not give it to them 😒 i had no other choice than to stay there since according to what i had booked and i had about   people obviou

In [6]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [7]:
remove_emoji = lambda x: emoji_pattern.sub(r'', x)

df['all_reviews'] = df.all_reviews.map(remove_emoji)

In [8]:
df.iloc[3].all_reviews

'  this is interesting  as it appears to be be under new ownership  it has a new paint job on the front  i have not taken time to speak with the new managers  in the past few years i know of people who have did extended vacations as the prices were reasonable  access in the local community offers many things  i live across the street    there is a bus route to centro up the hill and return from centro down the hill  they have a few rooms for family or long term  i have seen buses unload and many people take over the hotel in the past  time will tell     comfortable and decent  it is   minutes from the beach nthe water heater does not work and neither does the internet  but good for spending the night    very good cows in guadalajara    very good    change of administration and they did not make the deposit valid according to why the past administration did not give it to them  i had no other choice than to stay there since according to what i had booked and i had about   people obvious

In [9]:
df.iloc[28].all_reviews

'  jungle between the city  very exuberant place          '

In [10]:
df.iloc[14].all_reviews

'  good service and comfortable price                  '

In [11]:
# remove excessive spaces:
remove_spaces = lambda x: re.sub('        ',' ',x)
remove_spaces2 = lambda x: re.sub('   ', ' ', x)

df['all_reviews'] = df.all_reviews.map(remove_spaces).map(remove_spaces2)

In [12]:
df.iloc[28].all_reviews

'  jungle between the city  very exuberant place '

In [13]:
df[df['all_reviews'].str.len() <= 80]

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
12,OPERATIONAL,"{'lat': 20.6314507, 'lng': -105.2229678}",https://maps.gstatic.com/mapfiles/place_api/ic...,plaza central sur 150,NaN,NaN,ChIJzf0oz2NFIYQRuvlEAs6g2Fg,"{'compound_code': 'JQJG+HR Puerto Vallarta, Ja...",5.0,ChIJzf0oz2NFIYQRuvlEAs6g2Fg,...,plaza central sur 150,5.0,"[{'author_name': 'Sebastian Yañez', 'author_ur...",1.0,NaN,[5],quiet humble and very safe neighborhood,NaN,20.631451,-105.222968
14,OPERATIONAL,"{'lat': 20.6336142, 'lng': -105.2261243}",https://maps.gstatic.com/mapfiles/place_api/ic...,Apartamentos Lisboa,{'open_now': False},"[{'height': 3456, 'html_attributions': ['<a hr...",ChIJT--Qj2RFIYQRCkuXi7C7zKI,"{'compound_code': 'JQMF+CH Puerto Vallarta, Ja...",5.0,ChIJT--Qj2RFIYQRCkuXi7C7zKI,...,Apartamentos Lisboa,5.0,"[{'author_name': 'Joseluis Avila', 'author_url...",6.0,NaN,"[5, 5, 5, 5, 5]",good service and comfortable price,NaN,20.633614,-105.226124
16,OPERATIONAL,"{'lat': 20.6387746, 'lng': -105.22502}",https://maps.gstatic.com/mapfiles/place_api/ic...,Depa vallarta,NaN,NaN,ChIJGeUor59FIYQRJ4VgrqfrkuU,"{'compound_code': 'JQQF+GX Puerto Vallarta, Ja...",5.0,ChIJGeUor59FIYQRJ4VgrqfrkuU,...,Depa vallarta,5.0,"[{'author_name': 'Mr Vallarta', 'author_url': ...",1.0,NaN,[5],without fear of life,NaN,20.638775,-105.225020
25,OPERATIONAL,"{'lat': 20.6550001, 'lng': -105.2198112}",https://maps.gstatic.com/mapfiles/place_api/ic...,Petra Glass Hostal,NaN,"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJMSEIsB1PIYQRSGnL5x7Bxi8,"{'compound_code': 'MQ4J+23 Puerto Vallarta, Ja...",5.0,ChIJMSEIsB1PIYQRSGnL5x7Bxi8,...,Petra Glass Hostal,5.0,"[{'author_name': 'Petra Glass', 'author_url': ...",1.0,NaN,[5],very clean and inexpensive,NaN,20.655000,-105.219811
28,OPERATIONAL,"{'lat': 20.6538149, 'lng': -105.2254394}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Río Cuale,NaN,NaN,ChIJZyLH6opFIYQRgw11fW6Zndw,"{'compound_code': 'MQ3F+GR Puerto Vallarta, Ja...",5.0,ChIJZyLH6opFIYQRgw11fW6Zndw,...,Hotel Río Cuale,5.0,"[{'author_name': 'Fernando Martiñon', 'author_...",2.0,NaN,"[5, 5]",jungle between the city very exuberant place,NaN,20.653815,-105.225439
29,OPERATIONAL,"{'lat': 20.6533935, 'lng': -105.2253539}",https://maps.gstatic.com/mapfiles/place_api/ic...,Nora,NaN,"[{'height': 632, 'html_attributions': ['<a hre...",ChIJVVVVVYFFIYQR5ZAO5E7UWkM,"{'compound_code': 'MQ3F+9V Puerto Vallarta, Ja...",4.2,ChIJVVVVVYFFIYQR5ZAO5E7UWkM,...,Nora,4.2,"[{'author_name': 'Jesus Reyna', 'author_url': ...",5.0,NaN,"[5, 5, 5, 1, 5]",ermozo place i loved it i love this place ...,NaN,20.653393,-105.225354
31,OPERATIONAL,"{'lat': 20.65340699999999, 'lng': -105.2253316}",https://maps.gstatic.com/mapfiles/place_api/ic...,Puerto Vallarta,NaN,"[{'height': 1280, 'html_attributions': ['<a hr...",ChIJJ7hW4RNFIYQReKCg-JvCd5w,"{'compound_code': 'MQ3F+9V Puerto Vallarta, Ja...",5.0,ChIJJ7hW4RNFIYQReKCg-JvCd5w,...,Puerto Vallarta,5.0,"[{'author_name': 'IXOTECUTLI ZIGEINER', 'autho...",1.0,NaN,[5],cool place pal shitter,NaN,20.653407,-105.225332
33,OPERATIONAL,"{'lat': 20.6597267, 'lng': -105.22686}",https://maps.gstatic.com/mapfiles/place_api/ic...,Iris,NaN,"[{'height': 2560, 'html_attributions': ['<a hr...",ChIJf23sZ49FIYQRE5ppLkSSQpE,"{'compound_code': 'MQ5F+V7 Puerto Vallarta, Ja...",4.7,ChIJf23sZ49FIYQRE5ppLkSSQpE,...,Iris,4.7,"[{'author_name': 'ANTONIO VILLALVAZO', 'author...",3.0,NaN,"[5, 5, 4]",super good,NaN,20.659727,-105.226860
46,OPERATIONAL,"{'lat': 20.6658674, 'lng': -105.2525124}",https://maps.gstatic.com/mapfiles/place_api/ic...,Condominio Suite Resort,NaN,"[{'height': 1944, 'html_attributions': ['<a hr...",ChIJ0WlxTrhFIYQRoKWZr6nJeaY,"{'compound_code': 'MP8W+8X Puerto Vallarta, Ja...",4.4,ChIJ0WlxTrhFIYQRoKWZr6nJeaY,...,Condominio Suite Resort,4.4,"[{'author_name': 'Rocio Benitez', 'author_url'...",5.0,NaN,"[5, 5, 3, 4, 5]",

In [14]:
df['all_reviews']

0        an excellent place has all the comforts  it ...
1        it was more than i expected  cheap  spacious...
2        just for one night it s alright  discreet  a...
3        this is interesting  as it appears to be be ...
4        i loved it here   the place is clean  pleasa...
                             ...                        
382      love this resort and hotel  best room ever  ...
383                                   really nice view  
384      new accommodations  but landscaping needs to...
385                                                cool 
386                   there was no beef the day i went  
Name: all_reviews, Length: 387, dtype: object

In [15]:
df.iloc[344].all_reviews

'  ok   '

In [16]:
df1 = df[df['all_reviews'].str.len() > 80]

In [17]:
df1.shape

(346, 26)

In [18]:
df1 = df1.reset_index(drop=True)
df1.tail()

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
341,OPERATIONAL,"{'lat': 20.6778627, 'lng': -105.2852463}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Tower 1 at Vidanta Nuevo Vallarta,{'open_now': True},"[{'height': 768, 'html_attributions': ['<a hre...",ChIJ4SJARyNEIYQRx5laF9dUiN0,"{'compound_code': 'MPH7+4W Nuevo Vallarta, Nay...",4.8,ChIJ4SJARyNEIYQRx5laF9dUiN0,...,Grand Luxxe Tower 1 at Vidanta Nuevo Vallarta,4.8,"[{'author_name': 'Diane Tillman', 'author_url'...",371.0,NaN,"[5, 5, 3, 5, 5]",great location to pools bar restaurants m...,NaN,20.677863,-105.285246
342,OPERATIONAL,"{'lat': 20.6851287, 'lng': -105.287574}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grupo Vidanta Headquarters,NaN,"[{'height': 1001, 'html_attributions': ['<a hr...",ChIJK_fqDKNBIYQRU-pYVJ--ZNI,"{'compound_code': 'MPP6+3X Nuevo Vallarta, Nay...",3.8,ChIJK_fqDKNBIYQRU-pYVJ--ZNI,...,Grupo Vidanta Headquarters,3.8,"[{'author_name': 'Elsy B', 'author_url': 'http...",29.0,https://www.vidanta.com/,"[1, 5, 5, 2, 5]",very disappointed it seems that every time ...,NaN,20.685129,-105.287574
343,OPERATIONAL,"{'lat': 20.6828908, 'lng': -105.2788986}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Tower 5,NaN,"[{'height': 2160, 'html_attributions': ['<a hr...",ChIJI9D-5CVEIYQRyClZqRFcuoM,"{'compound_code': 'MPMC+5C Nuevo Vallarta, Nay...",4.8,ChIJI9D-5CVEIYQRyClZqRFcuoM,...,Grand Luxxe Tower 5,4.8,"[{'author_name': 'Mary Mackey', 'author_url': ...",437.0,https://www.vidanta.com/web/nuevo-vallarta,"[5, 5, 5, 5, 5]",the rooms are opulent the roof top pool and...,NaN,20.682891,-105.278899
344,OPERATIONAL,"{'lat': 20.6769383, 'lng': -105.285772}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Punta Tower at Vidanta Nuevo Vallarta,{'open_now': True},"[{'height': 3480, 'html_attributions': ['<a hr...",ChIJ7zhNNCNEIYQRDyRrWYr8hLk,"{'compound_code': 'MPG7+QM Nuevo Vallarta, Nay...",4.9,ChIJ7zhNNCNEIYQRDyRrWYr8hLk,...,Grand Luxxe Punta Tower at Vidanta Nuevo Vallarta,4.9,"[{'author_name': 'John Bittle', 'author_url': ...",64.0,https://www.vidanta.com/en/web/nuevo-vallarta/...,"[5, 5, 5, 5, 5]",love this resort and hotel best room ever ...,NaN,20.676938,-105.285772
345,OPERATIONAL,"{'lat': 20.6843008, 'lng': -105.2780858}",https://maps.gstatic.com/mapfiles/place_api/ic...,Torre 6 The Grand Luxxe,NaN,"[{'height': 2992, 'html_attributions': ['<a hr...",ChIJo9we6EJFIYQRx9y4RoVO4Bs,"{'compound_code': 'MPMC+PQ Jarretaderas, Nayar...",4.6,ChIJo9we6EJFIYQRx9y4RoVO4Bs,...,Torre 6 The Grand Luxxe,4.6,"[{'author_name': 'Dana Nyborg', 'author_url': ...",15.0,NaN,"[4, 5, 3, 5, 5]",new accommodations but landscaping needs to...,NaN,20.684301,-105.278086


In [19]:
df1.describe(include='all')

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
count,346,346,346,346,176,336,346,345,346.000000,346,...,346,346.000000,346,346.000000,256,346,346,1.0,346.000000,346.000000
unique,1,345,4,343,2,336,346,338,NaN,346,...,343,NaN,346,NaN,250,193,346,NaN,NaN,NaN
top,OPERATIONAL,"{'lat': 20.65340699999999, 'lng': -105.2253316}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hyatt Ziva Puerto Vallarta,{'open_now': True},"[{'height': 1152, 'html_attributions': ['<a hr...",ChIJ5-GK72dFIYQRMkfyY-4Ntu0,"{'compound_code': 'HQM4+WF Puerto Vallarta, Ja...",NaN,ChIJ5-GK72dFIYQRMkfyY-4Ntu0,...,Hyatt Ziva Puerto Vallarta,NaN,"[{'author_name': 'Sk James', 'author_url': 'ht...",NaN,https://www.vidanta.com/web/nuevo-vallarta,"[5, 5, 5, 5, 5]",great little hotel the staff are friendly ...,NaN,NaN,NaN
freq,346,2,324,2,136,1,1,2,NaN,1,...,2,NaN,1,NaN,3,65,1,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.389595,NaN,...,NaN,4.388439,NaN,412.187861,NaN,NaN,NaN,2.0,20.620304,-105.241177
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.499050,NaN,...,NaN,0.498443,NaN,899.480495,NaN,NaN,NaN,NaN,0.034150,0.016527
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,...,NaN,1.000000,NaN,1.000000,NaN,NaN,NaN,2.0,20.532667,-105.291951
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.200000,NaN,...,NaN,4.200000,NaN,11.000000,NaN,NaN,NaN,2.0,20.600183,-105.246052
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,NaN,...,NaN,4.500000,NaN,46.000000,NaN,NaN,NaN,2.0,20.614583,-105.236044
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.700000,NaN,...,NaN,4.700000,NaN,286.000000,NaN,NaN,NaN,2.0,20.643411,-105.231297


In [20]:
df1[df1['rating_x']<=3]

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
57,OPERATIONAL,"{'lat': 20.6590553, 'lng': -105.2469}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Villa Varadero,NaN,"[{'height': 2448, 'html_attributions': ['<a hr...",ChIJOdpzlbxFIYQRI-mzY60HRcQ,"{'compound_code': 'MQ53+J6 Puerto Vallarta, Ja...",2.7,ChIJOdpzlbxFIYQRI-mzY60HRcQ,...,Hotel Villa Varadero,2.7,"[{'author_name': 'xaviere pedroxa lopex', 'aut...",3.0,NaN,"[1, 4, 3]",there is an error a maps guides you to a v...,NaN,20.659055,-105.246900
130,OPERATIONAL,"{'lat': 20.6263699, 'lng': -105.2281492}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Vallarta,NaN,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJ093DB2dFIYQRYxq91vgbkJw,"{'compound_code': 'JQGC+GP Puerto Vallarta, Ja...",2.6,ChIJ093DB2dFIYQRYxq91vgbkJw,...,Hotel Vallarta,2.6,"[{'author_name': 'Kimberly Tyner', 'author_url...",7.0,NaN,"[1, 3, 1, 3, 4]",cruel people good place to rest only that t...,NaN,20.626370,-105.228149
142,OPERATIONAL,"{'lat': 20.613498, 'lng': -105.2328524}",https://maps.gstatic.com/mapfiles/place_api/ic...,The hostal spot,NaN,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJ28z2UUVFIYQRiW3MK5XgGnc,"{'compound_code': 'JQ78+9V Puerto Vallarta, Ja...",2.6,ChIJ28z2UUVFIYQRiW3MK5XgGnc,...,The hostal spot,2.6,"[{'author_name': 'Karen gudgel', 'author_url':...",7.0,NaN,"[1, 5, 5, 1, 1]",no hot water and the electricity was only a ...,NaN,20.613498,-105.232852
158,OPERATIONAL,"{'lat': 20.6131722, 'lng': -105.232317}",https://maps.gstatic.com/mapfiles/place_api/ic...,Departamento en el corazón de Vallarta,NaN,"[{'height': 2592, 'html_attributions': ['<a hr...",ChIJA7LxQ0VFIYQRiylo05bpyrM,"{'compound_code': 'JQ79+73 Puerto Vallarta, Ja...",1.0,ChIJA7LxQ0VFIYQRiylo05bpyrM,...,Departamento en el corazón de Vallarta,1.0,"[{'author_name': 'Addy Munguia', 'author_url':...",2.0,NaN,"[1, 1]",the business has been closed for years and g...,NaN,20.613172,-105.232317
279,OPERATIONAL,"{'lat': 20.5916992, 'lng': -105.24157}",https://maps.gstatic.com/mapfiles/place_api/ic...,Casa Destination Of Health Resort,NaN,"[{'height': 4128, 'html_attributions': ['<a hr...",ChIJrZeSN7ZaIYQRvhiC-HiH6lw,"{'compound_code': 'HQR5+M9 Puerto Vallarta, Ja...",1.8,ChIJrZeSN7ZaIYQRvhiC-HiH6lw,...,Casa Destination Of Health Resort,1.8,"[{'author_name': 'Bianca Ferratt', 'author_url...",4.0,http://www.casadestinationofhealth.mxhotel.site/,"[1, 1, 4, 1]",i made a reservation and the address does no...,NaN,20.591699,-105.241570
313,OPERATIONAL,"{'lat': 20.5483213, 'lng': -105.2731479}",https://maps.gstatic.com/mapfiles/place_api/ic...,Gran park royal,NaN,"[{'height': 943, 'html_attributions': ['<a hre...",ChIJVSghLVhbIYQRd69si_Ia1-U,"{'compound_code': 'GPXG+8P Aguacate, Jalisco, ...",2.7,ChIJVSghLVhbIYQRd69si_Ia1-U,...,Gran park royal,2.7,"[{'author_name': 'Kevin MX', 'author_url': 'ht...",3.0,NaN,"[4, 1, 3]",nice hotel good food pool is cold kids po...,NaN,20.548321,-105.273148


In [ ]:
# df1.to_csv(r'hotels_cleaned.csv', index=True, header=True, index_label=False)

In [21]:
df1 = df1[df1['rating_x']>3.5]

# NLP Round 1:

## Document-term Matrix with just Unigrams

In [22]:
vectorizer = CountVectorizer(stop_words='english', min_df=0.1, max_df=0.7)

doc_matrix = vectorizer.fit_transform(df1.all_reviews)

doc_matrix = pd.DataFrame(doc_matrix.toarray(), columns=vectorizer.get_feature_names())

In [23]:
doc_matrix.head()

,absolutely,amazing,area,attention,away,awesome,bad,bar,beach,beautiful,...,wait,walk,walking,want,water,way,went,wonderful,work,years
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,1,0,1,0,1,1
4,0,0,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Document-term Matrix with Bigrams and Unigrams

In [24]:
vectorizer_bi = CountVectorizer(ngram_range=(1,2), binary=True, stop_words='english', min_df=0.1, max_df=0.7)

doc_matrix_bi = vectorizer_bi.fit_transform(df1.all_reviews)

doc_matrix_bi = pd.DataFrame(doc_matrix_bi.toarray(), columns=vectorizer_bi.get_feature_names())

In [25]:
doc_matrix_bi.head()

,absolutely,amazing,area,attention,away,awesome,bad,bar,beach,beautiful,...,wait,walk,walking,want,water,way,went,wonderful,work,years
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,1,0,1,1
4,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
print(doc_matrix_bi.columns)

Index(['absolutely', 'amazing', 'area', 'attention', 'away', 'awesome', 'bad',
       'bar', 'beach', 'beautiful',
       ...
       'wait', 'walk', 'walking', 'want', 'water', 'way', 'went', 'wonderful',
       'work', 'years'],
      dtype='object', length=122)


## Document-term Matrix with Trigrams, Bigrams, and Unigrams

In [27]:
vectorizer_tri = CountVectorizer(ngram_range=(1,3), binary=True, stop_words='english', min_df=0.1, max_df=0.7)

doc_matrix_tri = vectorizer_bi.fit_transform(df1.all_reviews)

doc_matrix_tri = pd.DataFrame(doc_matrix_tri.toarray(), columns=vectorizer_tri.get_feature_names())

NotFittedError: Vocabulary not fitted or provided

## Top Words for Each Place of Lodging

In [28]:
word_doc = doc_matrix.transpose()

In [29]:
word_doc

,0,1,2,3,4,5,6,7,8,9,...,319,320,321,322,323,324,325,326,327,328
absolutely,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
amazing,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,1,0,0,3,0,0
area,0,0,0,0,2,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
attention,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
away,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
way,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
went,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
wonderful,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
work,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
top_hotels = {}
for hotel in word_doc.columns:
    top = word_doc[hotel].sort_values(ascending=False).head(30)
    top_hotels[hotel] = list(zip(top.index,top.values))

In [31]:
for hotel, top_words in top_hotels.items():
    print(hotel)
    print(', '.join([word for word, count in top_words[0:14]]))
    print('---')

0
wonderful, excellent, friendly, people, years, helpful, experience, family, feel, food, going, good, got, great
---
1
loved, beautiful, spacious, rooms, located, little, enjoyed, love, excellent, experience, family, feel, food, friendly
---
2
service, clean, night, just, good, quiet, comfortable, hot, excellent, location, large, got, experience, family
---
3
did, new, hotel, people, good, return, beach, close, rooms, time, water, private, stay, just
---
4
good, excellent, service, clean, area, stay, safe, location, just, people, quiet, easy, spacious, comfortable
---
5
nice, amazing, just, pool, come, people, time, stay, definitely, vacation, view, great, beautiful, friendly
---
6
bad, service, night, recommend, definitely, good, helpful, experience, family, feel, food, friendly, going, got
---
7
good, rooms, service, stay, money, town, like, ok, recommend, excellent, comfortable, time, loved, beautiful
---
8
large, just, safe, house, good, spacious, area, excellent, experience, fami

In [32]:
words = []
for hotel in word_doc.columns:
    top = [word for (word,count) in top_hotels[hotel]]
    for t in top:
        words.append(t)

In [33]:
Counter(words).most_common()

[('great', 296),
 ('friendly', 266),
 ('good', 263),
 ('hotel', 247),
 ('food', 246),
 ('feel', 220),
 ('just', 218),
 ('family', 209),
 ('going', 207),
 ('like', 202),
 ('helpful', 194),
 ('got', 192),
 ('experience', 186),
 ('excellent', 176),
 ('guests', 176),
 ('staff', 172),
 ('highly', 169),
 ('home', 168),
 ('hot', 167),
 ('location', 166),
 ('large', 162),
 ('house', 160),
 ('nice', 155),
 ('stay', 146),
 ('little', 145),
 ('pool', 145),
 ('clean', 137),
 ('service', 129),
 ('beautiful', 124),
 ('rooms', 121),
 ('room', 119),
 ('enjoyed', 118),
 ('beach', 116),
 ('located', 112),
 ('years', 108),
 ('amazing', 96),
 ('time', 93),
 ('best', 85),
 ('love', 82),
 ('recommend', 80),
 ('close', 79),
 ('view', 77),
 ('vallarta', 77),
 ('night', 72),
 ('people', 71),
 ('enjoy', 67),
 ('comfortable', 66),
 ('puerto', 63),
 ('price', 62),
 ('quiet', 61),
 ('really', 61),
 ('stayed', 61),
 ('definitely', 60),
 ('day', 60),
 ('area', 59),
 ('resort', 58),
 ('did', 57),
 ('restaurants', 57)

In [34]:
doc_matrix[doc_matrix['like']>=1]

,absolutely,amazing,area,attention,away,awesome,bad,bar,beach,beautiful,...,wait,walk,walking,want,water,way,went,wonderful,work,years
7,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
14,0,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43,1,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
45,2,5,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,1,2,0,0,0,0,0,1,1,0,...,0,0,0,0,1,1,2,0,0,0
312,0,0,0,0,0,0,1,0,2,0,...,0,1,1,2,1,0,2,0,0,0
313,1,1,0,0,1,0,0,0,1,0,...,2,0,1,0,0,0,0,0,0,2
315,0,0,0,0,1,0,0,1,0,0,...,0,0,1,1,0,1,2,0,0,0


## Add Stop Words

In [35]:
all_stops = ['going','got','like','did','didn','vallarta','puerto','really','super','puerto vallarta']

In [36]:
stop_words = text.ENGLISH_STOP_WORDS.union(all_stops)

In [37]:
vectorizer2 = CountVectorizer(stop_words=stop_words, min_df=0.1, max_df=0.7)

doc_matrix2 = vectorizer2.fit_transform(df1.all_reviews)

doc_matrix2 = pd.DataFrame(doc_matrix2.toarray(), columns=vectorizer2.get_feature_names())

In [38]:
vectorizer2_bi = CountVectorizer(stop_words=stop_words, min_df=0.1, max_df=0.7)

doc_matrix2_bi = vectorizer2_bi.fit_transform(df1.all_reviews)

doc_matrix2_bi = pd.DataFrame(doc_matrix2_bi.toarray(), columns=vectorizer2_bi.get_feature_names())

# TF-IDF

In [39]:
tfidf_vect = TfidfVectorizer(stop_words=stop_words, min_df=0.1, max_df=0.7)
x_tfidf = tfidf_vect.fit_transform(df1.all_reviews).toarray()
df_tfidf = pd.DataFrame(x_tfidf, columns=tfidf_vect.get_feature_names())

In [40]:
df_tfidf

,absolutely,amazing,area,attention,away,awesome,bad,bar,beach,beautiful,...,wait,walk,walking,want,water,way,went,wonderful,work,years
0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.591129,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.359883,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.190201,0.000000,...,0.0,0.0,0.0,0.0,0.132638,0.0,0.147848,0.000000,0.157689,0.160492
4,0.000000,0.000000,0.358741,0.21193,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,0.285851,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.283619,0.000000,0.170765,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
325,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.174148,0.000000,0.000000,0.217706,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
326,0.000000,0.472967,0.000000,0.00000,0.0,0.215565,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.203062,0.000000,0.000000
327,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.275079,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [41]:
tfidf_vect_bi = TfidfVectorizer(ngram_range=(1,2),stop_words=stop_words, min_df=0.1, max_df=0.7)
x_tfidf_bi = tfidf_vect_bi.fit_transform(df1.all_reviews).toarray()
df_tfidf_bi = pd.DataFrame(x_tfidf_bi, columns=tfidf_vect_bi.get_feature_names())

In [42]:
df_tfidf_bi

,absolutely,amazing,area,attention,away,awesome,bad,bar,beach,beautiful,...,wait,walk,walking,want,water,way,went,wonderful,work,years
0,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.591129,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.359883,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.190201,0.000000,...,0.0,0.0,0.0,0.0,0.132638,0.0,0.147848,0.000000,0.157689,0.160492
4,0.000000,0.000000,0.358741,0.21193,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,0.275314,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.273164,0.000000,0.164470,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
325,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.174148,0.000000,0.000000,0.217706,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
326,0.000000,0.472967,0.000000,0.00000,0.0,0.215565,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.203062,0.000000,0.000000
327,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.275079,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [43]:
df1.iloc[112].all_reviews

'  good place to hang out  although sometimes there are more children  but there is a pool that is only for adults at the top  recommended  it is a nice place to vacation and enjoy life  the best place for family vacations  well i have no complaints  '

In [44]:
df_tfidf_bi[df_tfidf_bi['bad']>0.3]

,absolutely,amazing,area,attention,away,awesome,bad,bar,beach,beautiful,...,wait,walk,walking,want,water,way,went,wonderful,work,years
6,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.765024,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
14,0.0,0.0,0.280300,0.000000,0.000000,0.0,0.303720,0.0,0.199732,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
21,0.0,0.0,0.000000,0.546151,0.000000,0.0,0.500865,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
104,0.0,0.0,0.231359,0.000000,0.000000,0.0,0.501378,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
107,0.0,0.0,0.000000,0.000000,0.164656,0.0,0.338694,0.0,0.111366,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.553975,0.0
126,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.468848,0.0,0.154161,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [45]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [46]:
def lsa_function(num_components, doc_text_matrix, vectorizer):
    lsa = TruncatedSVD(num_components)
    doc_topic = lsa.fit_transform(doc_text_matrix)
    print('Explained Variance:',lsa.explained_variance_ratio_)
    
    index = []
    for i in range(num_components):
        index.append(i)
    topic_word = pd.DataFrame(lsa.components_.round(3),
             index = index,
             columns = vectorizer.get_feature_names())
    
    print(display_topics(lsa, vectorizer.get_feature_names(), 15))
    
    return topic_word

In [47]:
def nmf_function(num_components, doc_text_matrix, vectorizer):
    nmf = NMF(num_components)
    doc_topic = nmf.fit_transform(doc_text_matrix)
    
    index = []
    for i in range(num_components):
        index.append(i)
    topic_word = pd.DataFrame(nmf.components_.round(3),
             index = index,
             columns = vectorizer.get_feature_names())
    
    print(display_topics(nmf, vectorizer.get_feature_names(), 15))
    
    return topic_word

In [48]:
lsa_function(4,doc_matrix,vectorizer)

Explained Variance: [0.1741311  0.06198132 0.05062777 0.04385782]

Topic  0
staff, great, hotel, room, pool, stay, resort, good, rooms, nice, clean, service, beach, amazing, food

Topic  1
resort, staff, amazing, food, day, experience, went, way, really, restaurant, house, property, service, bar, family

Topic  2
room, resort, rooms, didn, hotel, don, night, just, day, water, days, desk, did, bar, work

Topic  3
hotel, staff, stay, amazing, good, really, make, like, time, recommend, staying, money, friendly, desk, night
None


,absolutely,amazing,area,attention,away,awesome,bad,bar,beach,beautiful,...,wait,walk,walking,want,water,way,went,wonderful,work,years
0,0.037,0.143,0.045,0.018,0.034,0.034,0.038,0.061,0.147,0.102,...,0.036,0.041,0.031,0.056,0.057,0.042,0.051,0.037,0.040,0.025
1,0.055,0.235,-0.010,-0.014,-0.025,-0.010,-0.019,0.071,-0.055,0.033,...,0.039,-0.024,-0.023,-0.006,-0.042,0.092,0.094,0.027,0.002,0.006
2,-0.028,-0.151,-0.025,-0.007,-0.029,-0.042,0.029,0.056,-0.029,-0.209,...,0.004,-0.037,-0.018,0.033,0.075,0.010,-0.007,-0.061,0.054,0.002
3,0.011,0.092,-0.015,0.010,-0.005,-0.011,0.041,-0.001,-0.095,-0.089,...,0.021,-0.017,-0.025,0.024,-0.112,-0.010,0.027,-0.002,0.007,-0.006


In [49]:
lsa_function(4,doc_matrix_bi,vectorizer_bi)

Explained Variance: [0.09467173 0.04251031 0.03380742 0.02571969]

Topic  0
great, staff, stay, good, hotel, clean, pool, nice, friendly, rooms, room, beach, service, location, beautiful

Topic  1
amazing, puerto vallarta, puerto, beautiful, vallarta, experience, vacation, views, property, make, absolutely, recommend, went, best, house

Topic  2
food, experience, did, rooms, don, like, didn, room, check, time, better, service, work, bad, money

Topic  3
vallarta, puerto, puerto vallarta, close, stay, night, stayed, place stay, days, clean, super, door, water, hot, bed
None


,absolutely,amazing,area,attention,away,awesome,bad,bar,beach,beautiful,...,wait,walk,walking,want,water,way,went,wonderful,work,years
0,0.057,0.134,0.070,0.032,0.063,0.065,0.058,0.058,0.160,0.148,...,0.050,0.061,0.056,0.074,0.077,0.048,0.060,0.058,0.048,0.043
1,0.118,0.218,-0.011,0.000,-0.009,0.049,-0.015,0.034,-0.031,0.196,...,0.098,-0.020,-0.019,0.010,-0.056,0.023,0.113,0.061,0.050,0.029
2,0.057,-0.019,-0.061,0.004,-0.058,-0.089,0.107,0.061,-0.093,-0.226,...,0.024,-0.013,-0.039,0.070,0.023,0.062,0.034,-0.015,0.109,0.060
3,-0.021,-0.095,0.069,-0.060,0.033,-0.011,0.053,0.017,0.020,-0.361,...,0.051,0.040,0.078,0.053,0.090,-0.006,0.049,-0.141,0.047,-0.013


In [50]:
lsa_function(9,doc_matrix2,tfidf_vect)

Explained Variance: [0.17758058 0.0656914  0.05243456 0.04625118 0.03838672 0.03528584
 0.03017851 0.02707077 0.02487942]

Topic  0
staff, great, hotel, room, pool, stay, resort, good, rooms, nice, clean, service, beach, amazing, food

Topic  1
resort, staff, amazing, food, day, experience, went, way, restaurant, service, house, bar, property, family, vacation

Topic  2
room, resort, rooms, hotel, don, night, day, just, water, days, desk, better, work, bar, check

Topic  3
hotel, staff, stay, amazing, make, recommend, time, money, good, staying, night, desk, experience, guests, friendly

Topic  4
room, stay, beautiful, house, views, amazing, experience, time, best, just, view, nthe, make, recommend, night

Topic  5
staff, great, room, rooms, breakfast, good, food, helpful, friendly, bar, night, guests, clean, bed, price

Topic  6
hotel, great, restaurant, time, town, food, price, little, resort, enjoyed, old, right, breakfast, walk, staying

Topic  7
staff, pool, clean, nice, friendly,

,absolutely,amazing,area,attention,away,awesome,bad,bar,beach,beautiful,...,wait,walk,walking,want,water,way,went,wonderful,work,years
0,0.038,0.146,0.046,0.018,0.035,0.035,0.038,0.062,0.150,0.104,...,0.036,0.041,0.032,0.057,0.059,0.042,0.052,0.038,0.040,0.025
1,0.056,0.236,-0.010,-0.014,-0.026,-0.010,-0.019,0.073,-0.053,0.032,...,0.039,-0.026,-0.022,-0.005,-0.038,0.094,0.095,0.028,0.003,0.006
2,-0.031,-0.163,-0.023,-0.007,-0.028,-0.040,0.026,0.049,-0.011,-0.197,...,0.003,-0.037,-0.015,0.032,0.078,0.006,-0.008,-0.063,0.052,-0.001
3,0.013,0.095,-0.017,0.011,-0.007,-0.011,0.041,0.001,-0.090,-0.088,...,0.025,-0.022,-0.025,0.026,-0.109,-0.009,0.029,-0.004,0.011,-0.007
4,0.069,0.160,0.002,-0.015,-0.001,0.026,0.020,-0.035,-0.060,0.196,...,0.053,0.054,0.006,0.049,0.051,-0.018,-0.012,0.055,0.061,-0.018
5,-0.040,-0.038,-0.013,-0.003,-0.022,-0.036,-0.021,0.056,-0.196,-0.139,...,0.016,-0.016,-0.016,0.002,-0.022,0.012,-0.043,-0.006,-0.050,-0.033
6,0.004,-0.016,-0.045,-0.062,0.026,0.019,-0.018,-0.017,0.001,-0.091,...,-0.008,0.032,0.001,-0.031,0.006,0.020,0.002,0.011,0.000,-0.007
7,-0.039,-0.015,0.027,-0.071,-0.009,0.004,-0.039,0.028,-0.091,-0.263,...,-0.018,0.002,0.024,-0.000,-0.046,0.006,0.002,-0.071,-0.078,-0.028
8,0.018,-0.164,0.065,0.021,0.023,0.013,-0.007,0.054,-0.038,-0.023,...,0.008,0.033,0.035,-0.039,0.046,0.054,-0.016,0.013,-0.006,-0.007


In [51]:
nmf_function(14,doc_matrix2,tfidf_vect)


Topic  0
staff, amazing, friendly, helpful, food, went, bar, great, feel, come, guests, nthe, definitely, restaurant, way

Topic  1
resort, food, service, day, great, don, restaurant, family, way, bar, just, restaurants, property, feel, went

Topic  2
room, night, just, water, view, nthe, want, staff, bed, work, friendly, people, better, small, stayed

Topic  3
great, location, view, price, town, perfect, food, water, views, ocean, walk, away, walking, private, wonderful

Topic  4
time, amazing, experience, house, food, recommend, best, vacation, make, sure, trip, property, work, absolutely, highly

Topic  5
pool, day, location, quiet, time, just, days, restaurants, went, clean, nthe, vacation, family, love, ocean

Topic  6
hotel, time, little, friendly, days, restaurants, restaurant, breakfast, price, staying, money, clean, bad, recommend, nthe

Topic  7
good, service, excellent, food, attention, price, comfortable, ok, small, breakfast, restaurants, bad, visit, thing, years

Topic  

,absolutely,amazing,area,attention,away,awesome,bad,bar,beach,beautiful,...,wait,walk,walking,want,water,way,went,wonderful,work,years
0,0.000,1.646,0.017,0.000,0.000,0.013,0.000,0.329,0.000,0.000,...,0.182,0.000,0.000,0.187,0.000,0.202,0.382,0.086,0.000,0.000
1,0.062,0.059,0.000,0.009,0.000,0.000,0.000,0.420,0.015,0.059,...,0.111,0.000,0.000,0.004,0.122,0.429,0.251,0.000,0.054,0.079
2,0.000,0.000,0.000,0.009,0.000,0.000,0.222,0.160,0.000,0.000,...,0.127,0.000,0.078,0.460,0.597,0.000,0.000,0.000,0.310,0.000
3,0.000,0.000,0.041,0.000,0.166,0.018,0.000,0.000,0.000,0.000,...,0.000,0.171,0.161,0.000,0.254,0.095,0.000,0.139,0.000,0.000
4,0.658,2.119,0.096,0.043,0.000,0.184,0.355,0.000,0.000,0.038,...,0.367,0.239,0.000,0.388,0.000,0.037,0.495,0.256,0.668,0.157
5,0.007,0.083,0.131,0.000,0.058,0.002,0.000,0.000,0.093,0.000,...,0.000,0.000,0.122,0.086,0.174,0.000,0.236,0.000,0.000,0.160
6,0.000,0.000,0.000,0.000,0.142,0.101,0.173,0.000,0.083,0.094,...,0.000,0.000,0.000,0.111,0.000,0.000,0.000,0.032,0.066,0.000
7,0.000,0.000,0.101,0.358,0.000,0.000,0.149,0.000,0.000,0.000,...,0.000,0.000,0.029,0.072,0.000,0.000,0.000,0.000,0.000,0.124
8,0.383,0.000,0.488,0.013,0.101,0.058,0.023,0.356,0.000,0.000,...,0.111,0.184,0.176,0.037,0.000,0.291,0.060,0.103,0.000,0.000
9,0.208,0.207,0.000,0.285,0.197,0.305,0.000,0.000,0.000,4.964,...,0.153,0.000,0.000,0.000,0.211,0.000,0.000,0.832,0.000,0.008


In [52]:
df1.head()

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,name_y,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng
0,OPERATIONAL,"{'lat': 20.6939009, 'lng': -105.2742384}",https://maps.gstatic.com/mapfiles/place_api/ic...,casa LAS PIÑAS,{'open_now': False},"[{'height': 640, 'html_attributions': ['<a hre...",ChIJIeVYvYNGIYQRh-9bPVK3fCU,"{'compound_code': 'MPVG+H8 Jarretaderas, Nayar...",4.6,ChIJIeVYvYNGIYQRh-9bPVK3fCU,...,casa LAS PIÑAS,4.6,"[{'author_name': 'Jovita Garcia Fernandez', 'a...",8.0,https://www.facebook.com/departamentos.laspinas/,"[5, 5, 5, 5, 5]",an excellent place has all the comforts it ...,NaN,20.693901,-105.274238
1,OPERATIONAL,"{'lat': 20.6920731, 'lng': -105.2747303}",https://maps.gstatic.com/mapfiles/place_api/ic...,Villas Victoria Confort Jarretaderas,{'open_now': True},NaN,ChIJdSKPg3xHIYQRlZ0pmfgVPiQ,"{'compound_code': 'MPRG+R4 Jarretaderas, Nayar...",5.0,ChIJdSKPg3xHIYQRlZ0pmfgVPiQ,...,Villas Victoria Confort Jarretaderas,5.0,"[{'author_name': 'anahi chavez', 'author_url':...",1.0,https://www.villasvictoriacomfort.com/,[5],it was more than i expected cheap spacious...,NaN,20.692073,-105.274730
2,OPERATIONAL,"{'lat': 20.6939068, 'lng': -105.2763239}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Posada Alejandra,NaN,"[{'height': 349, 'html_attributions': ['<a hre...",ChIJB2Yha4NGIYQRE-GTtF_VSAU,"{'compound_code': 'MPVF+HF Jarretaderas, Nayar...",4.0,ChIJB2Yha4NGIYQRE-GTtF_VSAU,...,Hotel Posada Alejandra,4.0,"[{'author_name': 'Juan Carlos Lopez', 'author_...",54.0,https://www.google.com/url?sa=t&source=web&rct...,"[3, 4, 3, 3, 5]",just for one night it s alright discreet a...,NaN,20.693907,-105.276324
3,OPERATIONAL,"{'lat': 20.6241471, 'lng': -105.2243218}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hotel Azul Romero,{'open_now': True},"[{'height': 3120, 'html_attributions': ['<a hr...",ChIJFW5JHF5FIYQRogsYTW-KvMI,"{'compound_code': 'JQFG+M7 Puerto Vallarta, Ja...",3.9,ChIJFW5JHF5FIYQRogsYTW-KvMI,...,Hotel Azul Romero,3.9,"[{'author_name': 'Al Reust', 'author_url': 'ht...",11.0,http://www.azulromero.com/,"[3, 4, 5, 4, 1]",this is interesting as it appears to be be ...,NaN,20.624147,-105.224322
4,OPERATIONAL,"{'lat': 20.6225929, 'lng': -105.2232518}",https://maps.gstatic.com/mapfiles/place_api/ic...,Vacational Apartment Apartamento Vacacional De...,NaN,"[{'height': 1052, 'html_attributions': ['<a hr...",ChIJ9321RV5FIYQRwnf0qJkeIqU,"{'compound_code': 'JQFG+2M Puerto Vallarta, Ja...",4.8,ChIJ9321RV5FIYQRwnf0qJkeIqU,...,Vacational Apartment Apartamento Vacacional De...,4.8,"[{'author_name': 'Daniel Chilldres', 'author_u...",11.0,https://www.airbnb.mx/rooms/23351610,"[5, 4, 5, 5, 5]",i loved it here the place is clean pleasa...,NaN,20.622593,-105.223252


In [53]:
hotel_id = df1['place_id'].values

In [54]:
def nmf_HMatrix(num_components, doc_text_matrix, vectorizer):
    nmf = NMF(num_components)
    doc_topic = nmf.fit_transform(doc_text_matrix)
    
    idx = []
    for i in range(num_components):
        idx.append(i) 
    H = pd.DataFrame(doc_topic.round(3),
                    index = hotel_id,
                    columns = idx)
    return H

In [55]:
h1 = nmf_HMatrix(13,doc_matrix2,tfidf_vect)

In [56]:
h1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
ChIJIeVYvYNGIYQRh-9bPVK3fCU,0.018,0.0,0.011,0.004,0.009,0.000,0.006,0.002,0.000,0.026,0.034,0.000,0.087
ChIJdSKPg3xHIYQRlZ0pmfgVPiQ,0.000,0.0,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.086,0.000,0.134,0.000
ChIJB2Yha4NGIYQRE-GTtF_VSAU,0.000,0.0,0.051,0.003,0.000,0.023,0.000,0.171,0.083,0.000,0.215,0.000,0.354
ChIJFW5JHF5FIYQRogsYTW-KvMI,0.000,0.0,0.066,0.000,0.230,0.018,0.519,0.511,0.195,0.111,0.000,0.337,0.000
ChIJ9321RV5FIYQRwnf0qJkeIqU,0.000,0.0,0.000,0.001,0.000,0.012,0.000,0.503,0.370,0.018,0.249,0.000,0.672


In [57]:
doc_similarity_matrix = pd.DataFrame(cosine_similarity(h1))
doc_similarity_matrix.tail(15)

,0,1,2,3,4,5,6,7,8,9,...,319,320,321,322,323,324,325,326,327,328
314,0.258857,0.020213,0.366893,0.072276,0.223796,0.625434,0.073794,0.065758,0.131929,0.632084,...,0.300251,0.151014,0.153657,0.432128,0.250931,0.301521,0.091778,0.577108,0.372822,0.464206
315,0.175794,0.000000,0.253236,0.295358,0.343773,0.519155,0.313846,0.358938,0.372005,0.417890,...,0.594618,0.481137,0.493548,0.810740,0.622150,0.461251,0.649006,0.588102,0.544892,0.401853
316,0.293151,0.030989,0.524213,0.605954,0.584316,0.537649,0.634399,0.596070,0.642071,0.623812,...,0.280368,0.499811,0.502921,0.505693,0.522331,0.498357,0.483971,0.216057,0.624867,0.134809
317,0.526858,0.154442,0.769644,0.451670,0.798147,0.500694,0.642712,0.670346,0.601757,0.874480,...,0.304539,0.081153,0.115148,0.296373,0.127229,0.535847,0.343690,0.145312,0.328748,0.174417
318,0.107289,0.367576,0.071394,0.226065,0.057941,0.423103,0.055726,0.210142,0.083626,0.165733,...,0.904227,0.700288,0.532316,0.751290,0.664308,0.462158,0.482963,0.415084,0.478809,0.524134
319,0.154871,0.462889,0.180877,0.353501,0.178998,0.510720,0.195470,0.337028,0.269491,0.396270,...,1.000000,0.668354,0.459017,0.783064,0.671111,0.666110,0.502375,0.417002,0.662151,0.733069
320,0.207972,0.307986,0.124121,0.167623,0.047120,0.293405,0.121065,0.127838,0.060219,0.105293,...,0.668354,1.000000,0.753446,0.757908,0.883435,0.367068,0.569683,0.277056,0.712902,0.494291
321,0.122976,0.000000,0.119416,0.232453,0.051470,0.188484,0.069478,0.009604,0.040148,0.091942,...,0.459017,0.753446,1.000000,0.658324,0.695301,0.184194,0.744841,0.060033,0.672712,0.177267
322,0.178738,0.000000,0.254513,0.221258,0.215201,0.493528,0.268822,0.206459,0.321452,0.411779,...,0.783064,0.757908,0.658324,1.000000,0.835428,0.568919,0.591152,0.522587,0.782989,0.617239
323,0.230267,0.164657,0.093594,0.291424,0.038255,0.489546,0.145425,0.068893,0.162953,0.118698,...,0.671111,0.883435,0.695301,0.835428,1.000000,0.401430,0.499360,0.422617,0.836394,0.657650


In [58]:
df1.iloc[1].all_reviews

'  it was more than i expected  cheap  spacious and beautiful rooms  with retro  television  air conditioning  i loved '

In [154]:
df1.iloc[315].all_reviews

'  took a tour through and we went downtown i would think it s a great place to stay in  watch for time share recruitment  we stayed at the sea gardens for weeks before moving on to the grand mayan  n  i must say that the staff at the sea gardens went out of their way to make us feel at home even better  i would particularly like to mention gabriella and jonathan at the la rumba restaurant who made us feel well served with huge smiles and remembered us when we came back for breakfast after moving into the grand mayan all of us in our party agreed that the sea gardens was the place to be  we even met the cook   despite the rain your staff  lead by israel behind the bar  salvador  luis  johnathon aka senor slam and toni in the rumba or the go to christina who always smiles and knows the answer kept the good times going  thanks for another great visit  ron  older property  good location if you want to get away from the trapped in feeling of being at a resort  public transportation near by

# Word Embeddings

In [60]:
df1.iloc[0].all_reviews

'  an excellent place has all the comforts  it is a wonderful place and its people are very friendly   '

In [102]:
def w2v_tokenizer(document):
    stoplist = set('for a an it its very of the and to in'.split())
    texts = [word for word in document.lower().split() if word not in stoplist]
    
    return texts

In [103]:
df1.tail()

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng,w2vVector
341,OPERATIONAL,"{'lat': 20.6778627, 'lng': -105.2852463}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Tower 1 at Vidanta Nuevo Vallarta,{'open_now': True},"[{'height': 768, 'html_attributions': ['<a hre...",ChIJ4SJARyNEIYQRx5laF9dUiN0,"{'compound_code': 'MPH7+4W Nuevo Vallarta, Nay...",4.8,ChIJ4SJARyNEIYQRx5laF9dUiN0,...,4.8,"[{'author_name': 'Diane Tillman', 'author_url'...",371.0,NaN,"[5, 5, 3, 5, 5]",great location to pools bar restaurants m...,NaN,20.677863,-105.285246,-0.003904
342,OPERATIONAL,"{'lat': 20.6851287, 'lng': -105.287574}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grupo Vidanta Headquarters,NaN,"[{'height': 1001, 'html_attributions': ['<a hr...",ChIJK_fqDKNBIYQRU-pYVJ--ZNI,"{'compound_code': 'MPP6+3X Nuevo Vallarta, Nay...",3.8,ChIJK_fqDKNBIYQRU-pYVJ--ZNI,...,3.8,"[{'author_name': 'Elsy B', 'author_url': 'http...",29.0,https://www.vidanta.com/,"[1, 5, 5, 2, 5]",very disappointed it seems that every time ...,NaN,20.685129,-105.287574,-0.000811
343,OPERATIONAL,"{'lat': 20.6828908, 'lng': -105.2788986}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Tower 5,NaN,"[{'height': 2160, 'html_attributions': ['<a hr...",ChIJI9D-5CVEIYQRyClZqRFcuoM,"{'compound_code': 'MPMC+5C Nuevo Vallarta, Nay...",4.8,ChIJI9D-5CVEIYQRyClZqRFcuoM,...,4.8,"[{'author_name': 'Mary Mackey', 'author_url': ...",437.0,https://www.vidanta.com/web/nuevo-vallarta,"[5, 5, 5, 5, 5]",the rooms are opulent the roof top pool and...,NaN,20.682891,-105.278899,-0.003316
344,OPERATIONAL,"{'lat': 20.6769383, 'lng': -105.285772}",https://maps.gstatic.com/mapfiles/place_api/ic...,Grand Luxxe Punta Tower at Vidanta Nuevo Vallarta,{'open_now': True},"[{'height': 3480, 'html_attributions': ['<a hr...",ChIJ7zhNNCNEIYQRDyRrWYr8hLk,"{'compound_code': 'MPG7+QM Nuevo Vallarta, Nay...",4.9,ChIJ7zhNNCNEIYQRDyRrWYr8hLk,...,4.9,"[{'author_name': 'John Bittle', 'author_url': ...",64.0,https://www.vidanta.com/en/web/nuevo-vallarta/...,"[5, 5, 5, 5, 5]",love this resort and hotel best room ever ...,NaN,20.676938,-105.285772,-0.002048
345,OPERATIONAL,"{'lat': 20.6843008, 'lng': -105.2780858}",https://maps.gstatic.com/mapfiles/place_api/ic...,Torre 6 The Grand Luxxe,NaN,"[{'height': 2992, 'html_attributions': ['<a hr...",ChIJo9we6EJFIYQRx9y4RoVO4Bs,"{'compound_code': 'MPMC+PQ Jarretaderas, Nayar...",4.6,ChIJo9we6EJFIYQRx9y4RoVO4Bs,...,4.6,"[{'author_name': 'Dana Nyborg', 'author_url': ...",15.0,NaN,"[4, 5, 3, 5, 5]",new accommodations but landscaping needs to...,NaN,20.684301,-105.278086,-0.001328


In [104]:
df2 = df1.all_reviews.apply(w2v_tokenizer)

In [105]:
df2

0      [excellent, place, has, all, comforts, is, won...
1      [was, more, than, i, expected, cheap, spacious...
2      [just, one, night, s, alright, discreet, simpl...
3      [this, is, interesting, as, appears, be, be, u...
4      [i, loved, here, place, is, clean, pleasant, e...
                             ...                        
341    [great, location, pools, bar, restaurants, mus...
342    [disappointed, seems, that, every, time, you, ...
343    [rooms, are, opulent, roof, top, pool, restaur...
344    [love, this, resort, hotel, best, room, ever, ...
345    [new, accommodations, but, landscaping, needs,...
Name: all_reviews, Length: 329, dtype: object

In [106]:
w2v_tokenizer(df1.iloc[1].all_reviews)

['was',
 'more',
 'than',
 'i',
 'expected',
 'cheap',
 'spacious',
 'beautiful',
 'rooms',
 'with',
 'retro',
 'television',
 'air',
 'conditioning',
 'i',
 'loved']

In [107]:
def viable_words(words):
    viable_words = []
    for word in words:
        try:
            model[word]
            viable_words.append(word)
        except:
            pass
    return viable_words

In [108]:
df2 = df2.apply(viable_words)

In [109]:
df2

0      [excellent, place, has, all, comforts, is, won...
1      [was, more, than, i, expected, cheap, spacious...
2      [just, one, night, s, alright, discreet, simpl...
3      [this, is, interesting, as, appears, be, be, u...
4      [i, loved, here, place, is, clean, pleasant, e...
                             ...                        
341    [great, location, pools, bar, restaurants, mus...
342    [disappointed, seems, that, every, time, you, ...
343    [rooms, are, opulent, roof, top, pool, restaur...
344    [love, this, resort, hotel, best, room, ever, ...
345    [new, accommodations, but, landscaping, needs,...
Name: all_reviews, Length: 329, dtype: object

# Pre-trained Word2Vect Model

In [76]:
import os

# Setup nltk corpora path and Google Word2Vec location
google_vec_file = '/Users/lindsayread/Downloads/GoogleNews-vectors-negative300.bin'

In [77]:
model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [117]:
all_vectors = []
for word in df2.iloc[0]:
    all_vectors.append(model[word])

In [119]:
df2.iloc[0]

['excellent',
 'place',
 'has',
 'all',
 'comforts',
 'is',
 'wonderful',
 'place',
 'people',
 'are',
 'friendly']

In [120]:
np.mean(all_vectors)

-0.0049053575

In [93]:
def hotel_nlp_vector(viable_words):
    all_vectors = []
    for word in viable_words:
        all_vectors.append(model[word])
    return np.mean(all_vectors)

In [110]:
nlp_vectors = df2.apply(hotel_nlp_vector)

In [111]:
nlp_vectors

0     -0.004905
1      0.001456
2     -0.004143
3     -0.001768
4     -0.002578
         ...   
341   -0.003904
342   -0.001015
343   -0.003454
344   -0.002460
345   -0.001165
Name: all_reviews, Length: 329, dtype: float64

In [112]:
nlp_vectors.shape

(329,)

In [113]:
df1.shape

(329, 27)

In [114]:
df1['w2vVector'] = nlp_vectors

In [115]:
df1.describe(include='all')

,business_status,geometry,icon,name_x,opening_hours,photos,place_id,plus_code,rating_x,reference,...,rating_y,reviews,user_ratings_total_y,website,all_ratings,all_reviews,price_level_y,lat,lng,w2vVector
count,329,329,329,329,172,320,329,329,329.000000,329,...,329.000000,329,329.000000,249,329,329,1.0,329.000000,329.000000,329.000000
unique,1,328,4,326,2,320,329,323,NaN,329,...,NaN,329,NaN,243,177,329,NaN,NaN,NaN,NaN
top,OPERATIONAL,"{'lat': 20.65340699999999, 'lng': -105.2253316}",https://maps.gstatic.com/mapfiles/place_api/ic...,Hyatt Ziva Puerto Vallarta,{'open_now': True},"[{'height': 3840, 'html_attributions': ['<a hr...",ChIJ5-GK72dFIYQRMkfyY-4Ntu0,"{'compound_code': 'JQ37+97 Puerto Vallarta, Ja...",NaN,ChIJ5-GK72dFIYQRMkfyY-4Ntu0,...,NaN,"[{'author_name': 'Steve Edwards - Sitivi', 'au...",NaN,https://www.vidanta.com/web/nuevo-vallarta,"[5, 5, 5, 5, 5]",great little hotel the staff are friendly ...,NaN,NaN,NaN,NaN
freq,329,2,307,2,132,1,1,2,NaN,1,...,NaN,1,NaN,3,65,1,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.464438,NaN,...,4.463222,NaN,432.541033,NaN,NaN,NaN,2.0,20.620515,-105.241610,-0.002404
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.354044,NaN,...,0.353402,NaN,917.880000,NaN,NaN,NaN,NaN,0.034526,0.016557,0.001381
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.600000,NaN,...,3.600000,NaN,1.000000,NaN,NaN,NaN,2.0,20.532667,-105.291951,-0.010857
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.200000,NaN,...,4.200000,NaN,12.000000,NaN,NaN,NaN,2.0,20.600150,-105.246627,-0.003113
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,NaN,...,4.500000,NaN,54.000000,NaN,NaN,NaN,2.0,20.614356,-105.236168,-0.002124
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.700000,NaN,...,4.700000,NaN,308.000000,NaN,NaN,NaN,2.0,20.643852,-105.231399,-0.001469


In [116]:
df1.to_csv(r'nlp_embeddings.csv', index=True, header=True, index_label=False)

In [83]:
all_vectors = []
for i in df2.iloc[0]:
    all_vectors.append(model[i])
np.mean(all_vectors)

-0.0028293384

In [92]:
all_vectors = []
for i in df2.iloc[13]:
    all_vectors.append(model[i])
np.mean(all_vectors)

-0.0024749052

In [79]:
df2.iloc[0]

['an',
 'excellent',
 'place',
 'has',
 'all',
 'comforts',
 'it',
 'is',
 'wonderful',
 'place',
 'its',
 'people',
 'are',
 'very',
 'friendly']

In [123]:
from nltk.tokenize import TreebankWordTokenizer

In [ ]:
w2v

In [124]:
def get_embedded_review(review, embedding, tokenizer=TreebankWordTokenizer().tokenize):
    """
    Tokenizes a review and then returns average embedding vector across all the tokens
    
    :param review: <str> text of a single review
    :param embedding: keyed vectors representing word embeddings
    :param tokenizer: function that turns a long string of text into list of tokens
    :return: vector of the same dimension as embedding vectors, calculated as average of word vectors in review
    """
    tokens = tokenizer(review)
    # all text cleaning/pre-processing goes here
    # can put weight to tokens here... if you include "the", this word will have the same weight as all other words
    return np.mean([embedding[tok] for tok in tokens if tok in embedding], axis=0)

In [127]:
df1.iloc[0].all_reviews

'  an excellent place has all the comforts  it is a wonderful place and its people are very friendly   '

In [ ]:
stop_list = set('for a an it its very of the and to in'.split())
tree_tokens = [df1.all_reviews.apply(TreebankWordTokenizer().tokenize) if word not in stop_list]
tree_tokens.head()

In [136]:
tree_tokens = df1.all_reviews.apply(TreebankWordTokenizer().tokenize)
tree_tokens.head()

0    [an, excellent, place, has, all, the, comforts...
1    [it, was, more, than, i, expected, cheap, spac...
2    [just, for, one, night, it, s, alright, discre...
3    [this, is, interesting, as, it, appears, to, b...
4    [i, loved, it, here, the, place, is, clean, pl...
Name: all_reviews, dtype: object

In [139]:
def no_stops(words):
    stop_list = set('for a an it its very of the and to in'.split())
    return [word for word in words if word not in stop_list]

In [140]:
cleaned_tokens = tree_tokens.apply(no_stops)

In [141]:
cleaned_tokens.head()

0    [excellent, place, has, all, comforts, is, won...
1    [was, more, than, i, expected, cheap, spacious...
2    [just, one, night, s, alright, discreet, simpl...
3    [this, is, interesting, as, appears, be, be, u...
4    [i, loved, here, place, is, clean, pleasant, e...
Name: all_reviews, dtype: object

In [142]:
def get_embedded_review2(review, embedding):
    """
    Tokenizes a review and then returns average embedding vector across all the tokens
    
    :param review: <str> text of a single review
    :param embedding: keyed vectors representing word embeddings
    :param tokenizer: function that turns a long string of text into list of tokens
    :return: vector of the same dimension as embedding vectors, calculated as average of word vectors in review
    """
    #tokens = tokenizer(review)
    # all text cleaning/pre-processing goes here
    # can put weight to tokens here... if you include "the", this word will have the same weight as all other words
    return np.mean([embedding[tok] for tok in review if tok in embedding], axis=0)

In [144]:
embedded2 = cleaned_tokens.apply(viable_words)

In [145]:
embedded2

0      [excellent, place, has, all, comforts, is, won...
1      [was, more, than, i, expected, cheap, spacious...
2      [just, one, night, s, alright, discreet, simpl...
3      [this, is, interesting, as, appears, be, be, u...
4      [i, loved, here, place, is, clean, pleasant, e...
                             ...                        
341    [great, location, pools, bar, restaurants, mus...
342    [disappointed, seems, that, every, time, you, ...
343    [rooms, are, opulent, roof, top, pool, restaur...
344    [love, this, resort, hotel, best, room, ever, ...
345    [new, accommodations, but, landscaping, needs,...
Name: all_reviews, Length: 329, dtype: object

In [153]:
vectorized3 = embedded2.apply(hotel_nlp_vector)
vectorized3.head()

0   -0.004905
1    0.001456
2   -0.004143
3   -0.001768
4   -0.002578
Name: all_reviews, dtype: float64

In [148]:
def apply_vectors(list_of_words):
    list_of_vectors = []
    for word in list_of_words:
        list_of_vectors.append(model[word])
    return list_of_vectors

In [149]:
vectorized = embedded2.apply(apply_vectors)
vectorized.head()

0    [[-0.21289062, -0.0043029785, -0.18066406, -0....
1    [[0.026000977, -0.0018920898, 0.18554688, -0.0...
2    [[0.10107422, -0.0038146973, 0.018188477, 0.12...
3    [[0.109375, 0.140625, -0.03173828, 0.16601562,...
4    [[-0.22558594, -0.01953125, 0.09082031, 0.2373...
Name: all_reviews, dtype: object

In [151]:
vectorized.iloc[0]

[array([-2.12890625e-01, -4.30297852e-03, -1.80664062e-01, -7.56835938e-03,
         1.12792969e-01,  1.63085938e-01, -1.47094727e-02, -7.86132812e-02,
        -1.64062500e-01,  2.79296875e-01, -2.22656250e-01, -1.37329102e-02,
         1.45507812e-01, -4.56542969e-02,  5.85937500e-02,  1.85546875e-01,
         1.13769531e-01,  1.13769531e-01, -1.46484375e-01, -1.25122070e-02,
         8.78906250e-02,  1.44531250e-01, -6.88476562e-02,  1.68945312e-01,
         2.02148438e-01,  3.64685059e-03, -1.24023438e-01,  1.76757812e-01,
        -4.24804688e-02, -3.54003906e-03,  4.93164062e-02, -1.74804688e-01,
         2.53906250e-01,  1.32812500e-01,  2.63671875e-01,  2.87109375e-01,
        -1.98242188e-01, -9.81445312e-02, -1.18652344e-01, -1.43554688e-01,
         3.08593750e-01, -8.93554688e-02, -7.17163086e-03,  3.46679688e-02,
        -7.86132812e-02, -1.50390625e-01,  2.15820312e-01, -1.17675781e-01,
         2.85644531e-02, -2.41210938e-01,  1.22558594e-01,  1.28906250e-01,
        -9.2

In [150]:
embedded2_mean = np.mean(vectorized)
embedded2_mean.head()

TypeError: float() argument must be a string or a number, not 'list'

In [146]:
embedded2_mean = np.mean([model[word] for word in embedded2], axis=0)

ValueError: operands could not be broadcast together with shapes (11,300) (16,300) 